* 整体分析订单的主要来源，订单状态，订单的价格区间等
* 分析成功交易的订单主要分布在哪些地区，下单时间，渠道，商品名称等等
* 该数据缺乏用户的信息数据不做留存分析，用户特征等的分析


**提示💖：**

推荐练习项目👇：
🔑[用户订阅数据分析](https://www.heywhale.com/mw/project/62f7a14fb97cbff860f0a8f9)
🔑 [快手APP大学生用户分析](https://www.heywhale.com/mw/project/62fe414e6bb993e22e0cc0ed)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font='Microsoft YaHei',palette = 'cool',font_scale=1.3)
import matplotlib
matplotlib.rc('figure',figsize =(20,8))
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# 导入数据
data_8 = pd.read_excel('/home/mw/input/bilibili5461/bilibili2月8日订单.xlsx')
data_9 = pd.read_excel('/home/mw/input/bilibili5461/bilibili2月9日订单.xlsx')

In [3]:
print('data_8:',data_8.shape)
print('data_9:',data_9.shape)

data_8: (4912, 16)
data_9: (10551, 16)


In [4]:
# 合并数据
data = pd.concat([data_8,data_9],ignore_index = True)
print(data.shape)

(15463, 16)


## 1.处理数据部分

In [5]:
data.columns

Index(['商品编号', '商品名称', '商品价格', '票号', '座位', '联系地址', '渠道', '下单时间', '支付时间',
       '支付单号', '店铺名称', '店铺id', '订单状态', '售后状态', '是否海外购', '税费'],
      dtype='object')

In [6]:
print('去重前：',data.shape)
print('重复值：',data.duplicated().sum())
# 删除重复值
data.drop_duplicates(inplace = True)
print('去重后：',data.shape)

去重前： (15463, 16)
重复值： 89
去重后： (15374, 16)


In [7]:
print('统计空值:')
data.isnull().sum()

统计空值:


商品编号     0
商品名称     0
商品价格     0
票号       0
座位       0
联系地址     0
渠道       0
下单时间     0
支付时间     0
支付单号     0
店铺名称     0
店铺id     0
订单状态     0
售后状态     0
是否海外购    0
税费       0
dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15374 entries, 0 to 15462
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   商品编号    15374 non-null  int64         
 1   商品名称    15374 non-null  object        
 2   商品价格    15374 non-null  float64       
 3   票号      15374 non-null  object        
 4   座位      15374 non-null  object        
 5   联系地址    15374 non-null  object        
 6   渠道      15374 non-null  object        
 7   下单时间    15374 non-null  datetime64[ns]
 8   支付时间    15374 non-null  object        
 9   支付单号    15374 non-null  object        
 10  店铺名称    15374 non-null  object        
 11  店铺id    15374 non-null  int64         
 12  订单状态    15374 non-null  object        
 13  售后状态    15374 non-null  object        
 14  是否海外购   15374 non-null  object        
 15  税费      15374 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(11)
memory usage: 2.0+ MB


**处理时间**

In [9]:
d = ['day','hour']
for s in d:
    data['下单'+s] = getattr((pd.to_datetime(data['下单时间'])).dt,s)

**处理价格**

In [10]:
data['商品价格_g'] = pd.cut(data['商品价格'],bins = [-1,25,50,75,100,125,150,300,15000],
labels = ['25元内','25-50元内','50-75元内','75-100元内','100-125元内','125-150元内','150-300元内','>=300元'])

## 2.数据分析部分
* 先整体看订单的数据，后面在分别对比成功支付与取消订单的数据
* 稍微留意一下就可以发现，订单的主要来源渠道与成功交易的订单的主要渠道其实是不一样的

### 商品价格段分布

In [11]:
plt.subplot(1,2,1)
sns.countplot(x = '商品价格_g',data = data,palette = 'cool')
con = list(data.groupby('商品价格_g').商品价格.count().values)
for x,y in enumerate(con):
    plt.text(x-0.2,y,"%s" %y,size=14)
plt.ylabel('')
plt.xticks(rotation = 45)

plt.subplot(1,2,2)
sizes = data.groupby('商品价格_g').商品价格.count().values
labels = data.groupby('商品价格_g').商品价格.count().index
plt.pie(sizes ,labels = labels,autopct = '%1.1f%%',explode = [0.1,0,0,0,0,0,0,0],
startangle = 90,shadow = True,pctdistance = 0.8)
plt.show()

<Figure size 1440x576 with 2 Axes>

### 订单量

In [12]:
# 店铺订单量排名
plt.subplot(1,3,1)
sns.countplot(y = '店铺名称',data = data,palette = 'cool',order = data['店铺名称'].value_counts().index)
con = list(data['店铺名称'].value_counts().values)
for x,y in enumerate(con):
    plt.text(y,x,"%s" %y,size=14)
plt.ylabel('')
plt.xlabel('')
# plt.xticks(rotation = 90)
plt.title('店铺订单排名',loc = 'left')

plt.subplot(1,3,2)
sns.countplot(y = '下单day',data = data,palette = 'cool',order = data['下单day'].value_counts().index)
con = list(data['下单day'].value_counts().values)
for x,y in enumerate(con):
    plt.text(y,x,"%s" %y,size=14)
plt.ylabel('')
plt.yticks([0, 1],['9日下单','8日下单'])
plt.xlabel('')
plt.title('8-9日订单',loc = 'left')

plt.subplot(1,3,3)
sizes = data['下单day'].value_counts().values
labels = ['9日下单','8日下单']
plt.pie(sizes ,labels = labels,autopct = '%1.1f%%',explode = [0.1,0],
startangle = 90,shadow = True,pctdistance = 0.8)
plt.show()

<Figure size 1440x576 with 3 Axes>

### 订单状态

In [13]:
result_8 = data[data['下单day']== 8]
result_9 = data[data['下单day']== 9] 
plt.subplot(1,2,1)
sizes = result_8['订单状态'].value_counts().values
labels = result_8['订单状态'].value_counts().index
plt.pie(sizes ,labels = labels,autopct = '%1.1f%%',explode = [0.1,0,0,0],
startangle = 90,shadow = True)
plt.title('8日订单状态')

plt.subplot(1,2,2)
sizes = result_9['订单状态'].value_counts().values
labels = result_9['订单状态'].value_counts().index
plt.pie(sizes ,labels = labels,autopct = '%1.1f%%',explode = [0.1,0,0,0,0],
startangle = 90,shadow = True)
plt.title('9日订单状态')
plt.show()

<Figure size 1440x576 with 2 Axes>

### 下单渠道 Top 20

In [14]:
result = data['渠道'].value_counts()[:20].reset_index()
result.columns = ['渠道','total']

sns.barplot(y = '渠道',x= 'total',data = result,palette = 'cool')
con = list(result['total'])
for x,y in enumerate(con):
    plt.text(y,x,"%s" %y,size=12)
plt.ylabel('')
plt.xlabel('')
# plt.xticks(rotation = 90)
plt.title('订单渠道来源 top20',loc = 'left')
plt.show()

<Figure size 1440x576 with 1 Axes>

In [15]:
data['联系地址'] = data['联系地址'].replace('--','无')

In [16]:
data['地区']=data['联系地址'].str.split().str[0]

In [17]:
data.shape

(15374, 20)

In [18]:
data['订单状态'].value_counts()

待支付    7315
待收货    3261
已取消    2524
待发货    2193
已完成      81
Name: 订单状态, dtype: int64

### 已付款的订单主要来源哪些地区

In [19]:
result = data[data['订单状态'].isin(['待收货','待发货','已完成'])]
sns.countplot(x = result['地区'],data = result,order = result['地区'].value_counts().index,palette = 'cool')
con = list(result['地区'].value_counts().values)
for x,y in enumerate(con):
    plt.text(x-0.3,y,"%s" %y,size=14)
plt.xticks(rotation = 90)
plt.ylabel('')
plt.xlabel('')
plt.show()

<Figure size 1440x576 with 1 Axes>

In [20]:
from pyecharts.charts import Map,Page
from pyecharts import options as opts

In [21]:
result = data[data['订单状态'].isin(['待收货','待发货','已完成'])]
result = result['地区'].value_counts().reset_index()
result.columns = ['地区','total']
result['地区'] = result['地区'].str.replace('省','')
result['地区'] = result['地区'].str.replace('市','')
x_data = result['地区'].tolist()
y_data = result['total'].tolist()

m = Map()
m.add('',[list(z) for z in zip(x_data,y_data)],maptype = 'china',is_map_symbol_show=False)
m.set_series_opts(label_opts = opts.LabelOpts(is_show=True))
m.set_global_opts(title_opts = opts.TitleOpts(title = '交易成功的订单分布图'),
visualmap_opts = opts.VisualMapOpts(min_=result['total'].min(),max_=result['total'].max(),
range_color=['#27DAFE','#CA3DE3']))
m.render_notebook()

### 哪些地区取消的订单最多

In [22]:
result = data[data['订单状态']=='已取消']
sns.countplot(x = result['地区'],data = result,order = result['地区'].value_counts().index,color = '#CA3DE3')
con = list(result['地区'].value_counts().values)
for x,y in enumerate(con):
    plt.text(x-0.3,y,"%s" %y,size=14)
plt.xticks(rotation = 90)
plt.ylabel('')
plt.xlabel('')
plt.show()

<Figure size 1440x576 with 1 Axes>

### 交易成功的订单渠道来源

In [23]:
result = data[data['订单状态'].isin(['待收货','待发货','已完成'])]
result = result['渠道'].value_counts()[:20].reset_index()
result.columns = ['渠道','total']

sns.barplot(y = '渠道',x= 'total',data = result,palette = 'cool')
con = list(result['total'])
for x,y in enumerate(con):
    plt.text(y,x,"%s" %y,size=12)
plt.ylabel('')
plt.xlabel('')
# plt.xticks(rotation = 90)
plt.title('交易成功的订单渠道 top20',loc = 'left')
plt.show()

<Figure size 1440x576 with 1 Axes>

### 交易成功的订单——客户喜欢在哪个时段下单

In [24]:
result = data[data['订单状态'].isin(['待收货','待发货','已完成'])]
result = result.groupby('下单hour').商品编号.count().reset_index(name = 'total')
sns.lineplot(x = '下单hour',y = 'total',data = result)
plt.show()

<Figure size 1440x576 with 1 Axes>

### 交易成功的订单的其它信息

In [25]:
result = data[data['订单状态'].isin(['待收货','待发货','已完成'])]
plt.subplot(1,2,1)
sizes = result['下单day'].value_counts().values
labels = ['9日下单','8日下单']
plt.pie(sizes ,labels = labels,autopct = '%1.1f%%',explode = [0.1,0],
startangle = 90,shadow = True,pctdistance = 0.8)

plt.subplot(1,2,2)
sizes = result.groupby('商品价格_g').商品价格.count().values
labels = result.groupby('商品价格_g').商品价格.count().index
plt.pie(sizes ,labels = labels,autopct = '%1.1f%%',explode = [0,0,0,0,0,0,0,0.1],
shadow = True,pctdistance = 0.8)
plt.show()
plt.show()

<Figure size 1440x576 with 2 Axes>

### 成功交易订单 TOP 20 商品

In [26]:
result = data[data['订单状态'].isin(['待收货','待发货','已完成'])]
result = result['商品名称'].value_counts()[:20].reset_index()
result.columns = ['商品名称','total']

sns.barplot(y = '商品名称',x= 'total',data = result,palette = 'cool')
con = list(result['total'])
for x,y in enumerate(con):
    plt.text(y,x,"%s" %y,size=12)
plt.ylabel('')
plt.xlabel('')
# plt.xticks(rotation = 90)
plt.title('交易成功的订单商品名称 top20',loc = 'left')
plt.show()

<Figure size 1440x576 with 1 Axes>